In [182]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import csv

Since there aren't any table in wiki page <a>https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Bangalore</a>,we won't be able to use pandas.read_html method. \
Instead we will use BeautifulSoup to scrape wiki.

<h1>Web scraping using BeautifulSoup</h1>

In [183]:
site = requests.get('https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Bangalore').text 
soup = BeautifulSoup(site, 'lxml')

In [184]:
file = open('blr.csv', 'w')
csv_writer = csv.writer(file)
csv_writer.writerow(['Neighbourhood'])

15

In [185]:
mwcg_grps = soup.find_all(class_ = "mw-category-group")

length = len(mwcg_grps) 

for i in range(1, length):  # Iterating through all neighborhoods
    lists = mwcg_grps [i].find_all('a')
    for list in lists:
        ngh = list.get('title') # Name of the neighbourhood
        csv_writer.writerow([ngh]) 
file.close()

In [186]:
df_blr = pd.read_csv('blr.csv')

In [187]:
print(df_blr.shape)
df_blr.head()

(129, 1)


,Neighbourhood
0,Adugodi
1,"Agara, Bangalore"
2,Ananthnagar
3,Anjanapura
4,Arekere


In [188]:
df_blr.shape

(129, 1)

<h2>Finding latitude and longitude of bengaluru using nominatim</h2>

In [189]:
from geopy.geocoders import Nominatim 
address = 'Bangalore, India'

geolocator = Nominatim(user_agent="my_explorer")
loc = geolocator.geocode(address)
lat_blr = loc.latitude
long_blr = loc.longitude
print('The geograpical coordinate of Bangalore are {}, {}.'.format(lat_blr, long_blr))

The geograpical coordinate of Bangalore are 12.9791198, 77.5912997.


<h3>Getting latitude and longitude for all the neighborhood.</h3>

In [190]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install  folium
import folium

<h3>Now inserting latitude and longitude columns for each neighborhood in dataframe.</h3>

In [191]:
from geopy.extra.rate_limiter import RateLimiter
locator=Nominatim()

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df_blr['location'] = df_blr['Neighbourhood'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df_blr['point'] = df_blr['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df_blr[['latitude', 'longitude','altitude']] = pd.DataFrame(df_blr['point'].tolist(), index=df_blr.index)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


In [192]:
print(df_blr.shape)
df_blr.head()

(129, 6)


,Neighbourhood,location,point,latitude,longitude,altitude
0,Adugodi,"(Adugodi, South Zone, Bengaluru, Bangalore Sou...","(12.9428472, 77.6104155, 0.0)",12.942847,77.610416,0.0
1,"Agara, Bangalore","(Agara, Kanakapura taluk, Ramanagara district,...","(12.6201123, 77.4793074, 0.0)",12.620112,77.479307,0.0
2,Ananthnagar,None,None,NaN,NaN,NaN
3,Anjanapura,"(Anjanapura, Ramanagara taluk, Ramanagara dist...","(12.6847039, 77.2711718, 0.0)",12.684704,77.271172,0.0
4,Arekere,"(Arekere, Sakaleshapura taluk, Hassan district...","(12.8884342, 75.7794848, 0.0)",12.888434,75.779485,0.0


In [193]:
df_blr.drop(['point','altitude','location'],axis=1,inplace=True)

In [194]:
df_blr.dropna(inplace=True)

In [195]:
df_blr.shape

(115, 3)

# Exploring Bangalore using foilium and FourSquare APIs

<h3> Plotting map of Bangalore</h3>

In [196]:
# create map of Bangalore using latitude and longitude values
blr_map = folium.Map(location=[lat_blr, long_blr], zoom_start=11)   #Passing location info of Bangalore

# adding markers to map
for lat, lng, label in zip(df_blr['latitude'], df_blr['longitude'], df_blr['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#2186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(blr_map)  
    
blr_map

In [337]:
# define Foursquare Credentials and Version
#it has been hidden for security purposes.
CLIENT_ID = 'Hidden' # your Foursquare ID
CLIENT_SECRET = 'Hidden' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Hidden
CLIENT_SECRET:Hidden


In [198]:
def get_category_type(row):                 #function to assign category labels
    categories_list = row['Category']  
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [199]:
RADIUS=1000
LIMIT = 100

venues = []

for lat, long,neighborhood in zip(df_blr['latitude'], df_blr['longitude'],df_blr['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        RADIUS, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append(( 
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [200]:
print('There are {} uniques categories.'.format(len(venues_df['venue_category'].unique())))

There are 220 uniques categories.


In [149]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighbourhood', 'neighborhood_latitude', 'neighborhood_longitude', 'venue_name', 'venue_latitude', 'venue_longitude', 'venue_category']

print(venues_df.shape)
venues_df.head()

(2332, 7)


,Neighbourhood,neighborhood_latitude,neighborhood_longitude,venue_name,venue_latitude,venue_longitude,venue_category
0,Adugodi,12.942847,77.610416,PVR IMAX,12.934595,77.611321,Movie Theater
1,Adugodi,12.942847,77.610416,Lot Like Crêpes,12.936421,77.613284,Creperie
2,Adugodi,12.942847,77.610416,Zingron - Naga Kitchen,12.936271,77.615051,Indian Restaurant
3,Adugodi,12.942847,77.610416,Koramangala Social,12.935518,77.614097,Lounge
4,Adugodi,12.942847,77.610416,Tommy Hilfiger,12.934552,77.611347,Clothing Store


<h3> Performing One hot encoding on venue categories</h3>

In [202]:
# One hot encoding
blr_onehot = pd.get_dummies(venues_df[['venue_category']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe
blr_onehot['Neighbourhood'] = venues_df['Neighbourhood'] 

# Move neighborhood column to the first column
fixed_columns = [blr_onehot.columns[-1]] + blr_onehot.columns[:-1].values.tolist()
blr_onehot = blr_onehot[fixed_columns]
print(blr_onehot.shape)
blr_onehot.head()

(2332, 221)


,Neighbourhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Andhra Restaurant,Animal Shelter,Arcade,Art Gallery,...,Theater,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store
0,Adugodi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Adugodi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Adugodi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Adugodi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Adugodi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [216]:
blr_grouped = blr_onehot.groupby('Neighbourhood').mean().reset_index()
print(blr_grouped.shape)
blr_grouped.head()

(98, 221)


,Neighbourhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Andhra Restaurant,Animal Shelter,Arcade,Art Gallery,...,Theater,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store
0,Adugodi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.020833,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,Arekere,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,Austin Town,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.052632,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,BTM Layout,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.078431,0.0,0.0
4,"Bagalur, Bangalore Urban",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


<h2>Finding top 10 venues</h2>

In [204]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [312]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = blr_grouped['Neighbourhood']

for ind in np.arange(blr_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(blr_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adugodi,Indian Restaurant,Lounge,Dessert Shop,Café,Coffee Shop,Brewery,Bookstore,Multiplex,Donut Shop,Bus Station
1,Arekere,Resort,Women's Store,Eastern European Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant
2,Austin Town,Indian Restaurant,Hotel,Tea Room,Cocktail Bar,Café,Soccer Stadium,Donut Shop,Italian Restaurant,Indie Movie Theater,Clothing Store
3,BTM Layout,Indian Restaurant,Ice Cream Shop,Vegetarian / Vegan Restaurant,Snack Place,Bakery,Chinese Restaurant,Park,Sandwich Place,Café,Pizza Place
4,"Bagalur, Bangalore Urban",Food Truck,Memorial Site,French Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Financial or Legal Service,Field


# Performing clustering on venues

In [206]:
blr_clustering = blr_grouped.drop('Neighbourhood', 1)

In [338]:
from sklearn.metrics import silhouette_score

index1 = []
score1 = []

for kclusters in range(2,20) :
    blr_c = blr_clustering
    kmeans = KMeans(n_clusters = kclusters, init = 'k-means++', random_state = 0).fit_predict(blr_c)
    score2 = silhouette_score(blr_c, kmeans)
    index1.append(kclusters)
    score1.append(score2)

In [339]:
K_best = np.argmax(score1) + 2 # Finds the best K value
K_best

3

In [226]:
K = 4
blr_cl = blr_clustering
kmeans = KMeans(n_clusters = K, init = 'k-means++', random_state = 0).fit(blr_cl)

In [296]:
blr_clustered = blr_grouped.copy()

# add clustering labels
# blr_clustered["cluster_labels"] = kmeans.labels_
# blr_clustered.dropna(inplace=True)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
# blr_clustered = blr_clustered.join(neighborhoods_venues_sorted.set_index("Neighbourhood"), on="Neighbourhood")

print(blr_clustered.shape)
blr_clustered.head()

(98, 221)


,Neighbourhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Andhra Restaurant,Animal Shelter,Arcade,Art Gallery,...,Theater,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store
0,Adugodi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.020833,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,Arekere,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,Austin Town,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.052632,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,BTM Layout,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.078431,0.0,0.0
4,"Bagalur, Bangalore Urban",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [297]:
from geopy.extra.rate_limiter import RateLimiter
locator=Nominatim()

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
blr_clustered['location'] = blr_clustered['Neighbourhood'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
blr_clustered['point'] = blr_clustered['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
blr_clustered[['latitude', 'longitude','altitude']] = pd.DataFrame(blr_clustered['point'].tolist(), index=blr_clustered.index)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


In [298]:
blr_clustered.head()

,Neighbourhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Andhra Restaurant,Animal Shelter,Arcade,Art Gallery,...,Tram Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store,location,point,latitude,longitude,altitude
0,Adugodi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,"(Adugodi, South Zone, Bengaluru, Bangalore Sou...","(12.9428472, 77.6104155, 0.0)",12.942847,77.610416,0.0
1,Arekere,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,"(Arekere, Sakaleshapura taluk, Hassan district...","(12.8884342, 75.7794848, 0.0)",12.888434,75.779485,0.0
2,Austin Town,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,"(Austin Town, East Zone, Bengaluru, Bangalore ...","(12.9612736, 77.6152935, 0.0)",12.961274,77.615294,0.0
3,BTM Layout,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.078431,0.0,0.0,"(BTM Layout, South Zone, Bengaluru, Bangalore ...","(12.9151772, 77.6102821, 0.0)",12.915177,77.610282,0.0
4,"Bagalur, Bangalore Urban",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,"(Bagaluru, Bangalore North, Bangalore Urban, K...","(13.1331868, 77.6687093, 0.0)",13.133187,77.668709,0.0


In [299]:
blr_clustered.drop(['location','point','altitude'],axis=1,inplace=True)

In [309]:
# add clustering labels
blr_clustered["cluster_labels"] = kmeans.labels_
# blr_clustered.dropna(inplace=True)



In [300]:
blr_grouped.shape

(98, 221)

In [301]:
fixed_columns1 = [blr_clustered.columns[0]]+blr_clustered.columns[-2:].values.tolist() + blr_clustered.columns[1:-2].values.tolist()
blr_clustered = blr_clustered[fixed_columns1]

In [303]:
blr_clustered.head()

,Neighbourhood,latitude,longitude,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Andhra Restaurant,Animal Shelter,...,Theater,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store
0,Adugodi,12.942847,77.610416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.020833,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,Arekere,12.888434,75.779485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,Austin Town,12.961274,77.615294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.052632,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,BTM Layout,12.915177,77.610282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.078431,0.0,0.0
4,"Bagalur, Bangalore Urban",13.133187,77.668709,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [305]:
fixed_columns2=blr_clustered.columns[:3].values.tolist()+[blr_clustered.columns[-1]]+blr_clustered.columns[3:].values.tolist()
blr_clustered=blr_clustered[fixed_columns2]

In [306]:
blr_clustered.head()

,Neighbourhood,latitude,longitude,cluster_labels,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Andhra Restaurant,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store,cluster_labels
0,Adugodi,12.942847,77.610416,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.020833,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1
1,Arekere,12.888434,75.779485,2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,2
2,Austin Town,12.961274,77.615294,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.052632,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0
3,BTM Layout,12.915177,77.610282,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.078431,0.0,0.0,0
4,"Bagalur, Bangalore Urban",13.133187,77.668709,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1


In [307]:
blr_clustered.drop(blr_clustered.columns[4:].values.tolist(),axis=1,inplace=True)

In [310]:
blr_clustered.head()

,Neighbourhood,latitude,longitude,cluster_labels
0,Adugodi,12.942847,77.610416,1
1,Arekere,12.888434,75.779485,2
2,Austin Town,12.961274,77.615294,0
3,BTM Layout,12.915177,77.610282,0
4,"Bagalur, Bangalore Urban",13.133187,77.668709,1


In [314]:
neighbourhoods_venues_sorted.shape

(98, 11)

In [315]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
blr_clustered = blr_clustered.join(neighbourhoods_venues_sorted.set_index("Neighbourhood"), on="Neighbourhood")

In [317]:
blr_clustered.head()

,Neighbourhood,latitude,longitude,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adugodi,12.942847,77.610416,1,Indian Restaurant,Lounge,Dessert Shop,Café,Coffee Shop,Brewery,Bookstore,Multiplex,Donut Shop,Bus Station
1,Arekere,12.888434,75.779485,2,Resort,Women's Store,Eastern European Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Financial or Legal Service,Field,Fast Food Restaurant
2,Austin Town,12.961274,77.615294,0,Indian Restaurant,Hotel,Tea Room,Cocktail Bar,Café,Soccer Stadium,Donut Shop,Italian Restaurant,Indie Movie Theater,Clothing Store
3,BTM Layout,12.915177,77.610282,0,Indian Restaurant,Ice Cream Shop,Vegetarian / Vegan Restaurant,Snack Place,Bakery,Chinese Restaurant,Park,Sandwich Place,Café,Pizza Place
4,"Bagalur, Bangalore Urban",13.133187,77.668709,1,Food Truck,Memorial Site,French Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Financial or Legal Service,Field


# plotting the clusters on map.

In [320]:
map_clusters = folium.Map(location=[lat_blr, long_blr], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(blr_clustered['latitude'], blr_clustered['longitude'], blr_clustered['Neighbourhood'], blr_clustered['cluster_labels']):
    label = folium.Popup(str(poi) + ' (Cluster ' + str(cluster + 1) + ')', parse_html=True)
    map_clusters.add_child(
        folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7))
       
map_clusters

# Deciding place where indian restaurants can be opened in Bangalore.

In [336]:
blr_clustered[(blr_clustered['cluster_labels'] == 0) & (blr_merged['1st Most Common Venue']== 'Indian Restaurant')]

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,Neighbourhood,latitude,longitude,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Banaswadi,13.014162,77.651854,0,Indian Restaurant,Café,Department Store,Park,Arts & Crafts Store,Bus Station,Bistro,Kerala Restaurant,Tea Room,Intersection
10,Bommanahalli,12.908945,77.623904,0,Indian Restaurant,Clothing Store,Coffee Shop,Shop & Service,Chettinad Restaurant,Tea Room,Asian Restaurant,Café,Vegetarian / Vegan Restaurant,Flea Market
16,Chamrajpet,12.956997,77.563368,0,Indian Restaurant,Department Store,Coffee Shop,Fast Food Restaurant,Breakfast Spot,Multiplex,Park,Café,Shopping Mall,Big Box Store
17,Chickpet,12.968003,77.578642,0,Indian Restaurant,Theater,Historic Site,Food,Market,Flea Market,Bookstore,Snack Place,Miscellaneous Shop,South Indian Restaurant
20,"Cox Town, Bangalore",12.995483,77.620868,0,Indian Restaurant,Café,BBQ Joint,Middle Eastern Restaurant,Ice Cream Shop,Pizza Place,Bakery,Pub,Chinese Restaurant,Coffee Shop
22,Dollars Colony,13.042068,77.568015,0,Indian Restaurant,Department Store,Pizza Place,Snack Place,Shipping Store,Chinese Restaurant,Grocery Store,Paper / Office Supplies Store,Park,Accessories Store
27,"Gandhi Nagar, Bangalore",12.977153,77.578401,0,Indian Restaurant,Hotel,Seafood Restaurant,Racetrack,Bed & Breakfast,Café,Electronics Store,Asian Restaurant,Bookstore,Clothing Store
31,HSR Layout,12.911623,77.638862,0,Indian Restaurant,Café,Ice Cream Shop,Pizza Place,Snack Place,South Indian Restaurant,Gym,Liquor Store,Bakery,Fast Food Restaurant
43,Jeevanbheema Nagar,12.965818,77.657662,0,Indian Restaurant,Fast Food Restaurant,Chinese Restaurant,Café,Pizza Place,Smoke Shop,Gym,Korean Restaurant,Park,Coffee Shop
44,Jigani,12.785253,77.638093,0,Indian Restaurant,Hotel,Health Food Store,Women's Store,Eastern European Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Financial or Legal Service


In [340]:
blr_clustered[(blr_clustered['cluster_labels'] == 1) & (blr_merged['1st Most Common Venue']== 'Indian Restaurant')]

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,Neighbourhood,latitude,longitude,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adugodi,12.942847,77.610416,1,Indian Restaurant,Lounge,Dessert Shop,Café,Coffee Shop,Brewery,Bookstore,Multiplex,Donut Shop,Bus Station
19,Cooke Town,13.002785,77.624747,1,Café,Convenience Store,Department Store,Shopping Mall,Indian Restaurant,Grocery Store,Juice Bar,Lounge,Fast Food Restaurant,Movie Theater
24,Ejipura,12.945245,77.626914,1,Pizza Place,Café,Ice Cream Shop,Clothing Store,Indian Restaurant,Gym / Fitness Center,Golf Course,Department Store,Seafood Restaurant,Food Court
25,Electronic City,32.293317,-64.782372,1,Restaurant,Café,Italian Restaurant,Sandwich Place,Pub,Sushi Restaurant,Seafood Restaurant,Coffee Shop,Department Store,Pharmacy
32,"Hanumanthanagar, Bengaluru",13.060615,77.669273,1,Pizza Place,Café,Pool,Karnataka Restaurant,Business Service,Food,Flower Shop,Flea Market,Financial or Legal Service,Field
36,Horamavu,13.027331,77.660151,1,Department Store,Kerala Restaurant,Athletics & Sports,Andhra Restaurant,Badminton Court,Financial or Legal Service,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field
37,Hulimavu,12.877349,77.602803,1,Café,Indian Restaurant,Movie Theater,Electronics Store,Clothing Store,Fast Food Restaurant,South Indian Restaurant,Multiplex,Pizza Place,Dumpling Restaurant
38,Immadihalli,12.966399,77.762046,1,Fried Chicken Joint,Bakery,Restaurant,Women's Store,Electronics Store,Food & Drink Shop,Food,Flower Shop,Flea Market,Financial or Legal Service
45,Kaggadasapura,12.984671,77.679091,1,Food Truck,Restaurant,BBQ Joint,Gym,Asian Restaurant,Grocery Store,Department Store,Bridge,Flower Shop,Flea Market
47,Kamakshipalya,12.982362,77.522638,1,ATM,Breakfast Spot,Park,Bus Station,Campground,Event Space,Food & Drink Shop,Food,Flower Shop,Flea Market


In [352]:
blr_clustered[(blr_clustered['cluster_labels'] == 0)]

,Neighbourhood,latitude,longitude,cluster_labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Austin Town,12.961274,77.615294,0,Indian Restaurant,Hotel,Tea Room,Cocktail Bar,Café,Soccer Stadium,Donut Shop,Italian Restaurant,Indie Movie Theater,Clothing Store
3,BTM Layout,12.915177,77.610282,0,Indian Restaurant,Ice Cream Shop,Vegetarian / Vegan Restaurant,Snack Place,Bakery,Chinese Restaurant,Park,Sandwich Place,Café,Pizza Place
5,Banaswadi,13.014162,77.651854,0,Indian Restaurant,Café,Department Store,Park,Arts & Crafts Store,Bus Station,Bistro,Kerala Restaurant,Tea Room,Intersection
6,Basaveshwaranagar,12.993757,77.539132,0,Ice Cream Shop,Indian Restaurant,Fast Food Restaurant,Snack Place,Chinese Restaurant,Café,Pizza Place,Juice Bar,Liquor Store,Burger Joint
9,Bilekahalli,12.899593,77.610317,0,Department Store,Indian Restaurant,Ice Cream Shop,Tea Room,Park,Dessert Shop,Café,Breakfast Spot,Bar,Bakery
10,Bommanahalli,12.908945,77.623904,0,Indian Restaurant,Clothing Store,Coffee Shop,Shop & Service,Chettinad Restaurant,Tea Room,Asian Restaurant,Café,Vegetarian / Vegan Restaurant,Flea Market
11,Bommasandra,12.816244,77.691611,0,Indian Restaurant,Coffee Shop,Department Store,Chinese Restaurant,Sporting Goods Shop,Women's Store,Event Space,Food & Drink Shop,Food,Flower Shop
13,Byatarayanapura,13.062074,77.596392,0,Indian Restaurant,Ice Cream Shop,Brewery,Cosmetics Shop,Pizza Place,Construction & Landscaping,Coffee Shop,Chinese Restaurant,Electronics Store,Café
15,Carmelaram,12.911200,77.706513,0,South Indian Restaurant,Indian Restaurant,Tennis Court,Train Station,Restaurant,Flower Shop,Flea Market,Financial or Legal Service,Eastern European Restaurant,Field
16,Chamrajpet,12.956997,77.563368,0,Indian Restaurant,Department Store,Coffee Shop,Fast Food Restaurant,Breakfast Spot,Multiplex,Park,Café,Shopping Mall,Big Box Store
